In [1]:
import pandas as pd

In [5]:
df= pd.read_csv('C:\Git\mymacinelearning\mymachinelearning\laptops.csv')

In [ ]:
# Normalize the column names
df.columns = df.columns.str.lower().str.replace(' ', '_')


# Display the selected columns to verify
df.columns

In [7]:
# Selecting the required columns
df= df[['ram', 'storage', 'screen', 'final_price']]

In [ ]:
# Check for missing values in the selected columns
df_missing_values = df.isnull().sum()
df_missing_values


In [15]:
# Calculate the median for the 'ram' column
ram_median=df[['ram']].median()
ram_median




ram    16.0
dtype: float64

In [18]:
#Question 3

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Step 1: Shuffle the data
df_shuffled = df.sample(frac=1, random_state=42)

# Step 2: Split the data into train/validation/test sets (60%/20%/20%)
train_df, temp_df = train_test_split(df_shuffled, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Step 3: Calculate the mean of the 'screen' column using training data
screen_mean = train_df['screen'].mean()

# Option 1: Fill missing values with 0
train_df_0 = train_df.fillna({'screen': 0})
val_df_0 = val_df.fillna({'screen': 0})

# Option 2: Fill missing values with the mean of the training set
train_df_mean = train_df.fillna({'screen': screen_mean})
val_df_mean = val_df.fillna({'screen': screen_mean})

# Prepare the data for training
X_train_0, y_train_0 = train_df_0[['ram', 'storage', 'screen']], train_df_0['final_price']
X_val_0, y_val_0 = val_df_0[['ram', 'storage', 'screen']], val_df_0['final_price']

X_train_mean, y_train_mean = train_df_mean[['ram', 'storage', 'screen']], train_df_mean['final_price']
X_val_mean, y_val_mean = val_df_mean[['ram', 'storage', 'screen']], val_df_mean['final_price']

# Train linear regression models for each option
model_0 = LinearRegression()
model_0.fit(X_train_0, y_train_0)

model_mean = LinearRegression()
model_mean.fit(X_train_mean, y_train_mean)

# Make predictions
val_pred_0 = model_0.predict(X_val_0)
val_pred_mean = model_mean.predict(X_val_mean)

# Calculate RMSE for each model
rmse_0 = round(np.sqrt(mean_squared_error(y_val_0, val_pred_0)), 2)
rmse_mean = round(np.sqrt(mean_squared_error(y_val_mean, val_pred_mean)), 2)

rmse_0, rmse_mean


(675.08, 675.16)

In [19]:
#Question 4
#Now let's train a regularized linear regression.
#For this question, fill the NAs with 0.
#Try different values of r from this list: [0, 0.01, 0.1, 1, 5, 10, 100].
#Use RMSE to evaluate the model on the validation dataset.
#Round the RMSE scores to 2 decimal digits.
#Which r gives the best RMSE?

from sklearn.linear_model import Ridge

# List of regularization strengths
r_values = [0, 0.01, 0.1, 1, 5, 10, 100]

# Prepare the train and validation sets (using NAs filled with 0)
X_train, y_train = train_df_0[['ram', 'storage', 'screen']], train_df_0['final_price']
X_val, y_val = val_df_0[['ram', 'storage', 'screen']], val_df_0['final_price']

# Store results for each value of r
rmse_results = {}

# Train Ridge regression models for each regularization strength
for r in r_values:
    model_ridge = Ridge(alpha=r)
    model_ridge.fit(X_train, y_train)
    val_pred = model_ridge.predict(X_val)
    rmse = round(np.sqrt(mean_squared_error(y_val, val_pred)), 2)
    rmse_results[r] = rmse

# Finding the best r value
best_r = min(rmse_results, key=rmse_results.get)

rmse_results, best_r


({0: 675.08,
  0.01: 675.08,
  0.1: 675.08,
  1: 675.08,
  5: 675.08,
  10: 675.08,
  100: 675.01},
 100)

In [20]:
#Question 5
# List of seed values to test
seed_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Store RMSE results for each seed
rmse_seed_results = []

# Loop through each seed and calculate RMSE
for seed in seed_values:
    # Shuffle and split the data using the current seed
    df_shuffled_seed = df.sample(frac=1, random_state=seed)
    train_df_seed, temp_df_seed = train_test_split(df_shuffled_seed, test_size=0.4, random_state=seed)
    val_df_seed, _ = train_test_split(temp_df_seed, test_size=0.5, random_state=seed)
    
    # Fill missing values with 0
    train_df_seed = train_df_seed.fillna({'screen': 0})
    val_df_seed = val_df_seed.fillna({'screen': 0})
    
    # Prepare the train and validation sets
    X_train_seed, y_train_seed = train_df_seed[['ram', 'storage', 'screen']], train_df_seed['final_price']
    X_val_seed, y_val_seed = val_df_seed[['ram', 'storage', 'screen']], val_df_seed['final_price']
    
    # Train the linear regression model
    model_seed = LinearRegression()
    model_seed.fit(X_train_seed, y_train_seed)
    
    # Make predictions and calculate RMSE
    val_pred_seed = model_seed.predict(X_val_seed)
    rmse_seed = np.sqrt(mean_squared_error(y_val_seed, val_pred_seed))
    rmse_seed_results.append(rmse_seed)

# Calculate the standard deviation of RMSE scores
rmse_std = round(np.std(rmse_seed_results), 3)
rmse_std


23.888

In [21]:
# Split the data using seed 9
df_shuffled_seed9 = df.sample(frac=1, random_state=9)
train_df_seed9, temp_df_seed9 = train_test_split(df_shuffled_seed9, test_size=0.4, random_state=9)
val_df_seed9, test_df_seed9 = train_test_split(temp_df_seed9, test_size=0.5, random_state=9)

# Combine train and validation datasets
combined_train_val_df = pd.concat([train_df_seed9, val_df_seed9])

# Fill missing values with 0
combined_train_val_df = combined_train_val_df.fillna({'screen': 0})
test_df_seed9 = test_df_seed9.fillna({'screen': 0})

# Prepare train/validation and test sets
X_train_val = combined_train_val_df[['ram', 'storage', 'screen']]
y_train_val = combined_train_val_df['final_price']
X_test = test_df_seed9[['ram', 'storage', 'screen']]
y_test = test_df_seed9['final_price']

# Train the Ridge regression model with r=0.001
model_ridge_001 = Ridge(alpha=0.001)
model_ridge_001.fit(X_train_val, y_train_val)

# Make predictions on the test set and calculate RMSE
test_pred = model_ridge_001.predict(X_test)
rmse_test = round(np.sqrt(mean_squared_error(y_test, test_pred)), 2)

rmse_test


552.86